<a href="https://www.kaggle.com/code/pranavmpillai/heartdisease-prediction-from-audio?scriptVersionId=142877465" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install kaggle

In [ ]:
import os
os.getcwd()

In [ ]:
os.listdir()

In [ ]:
# os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/kaggle'

In [ ]:
# !pip install kaggle

In [ ]:
# os.chdir('/content/drive/MyDrive/gdrive')

In [ ]:
# !kaggle datasets download -d swapnilpanda/heart-sound-database

In [ ]:
os.listdir('/kaggle/input/heart-sound-database/heart_sound')

In [ ]:
# zip_path = '/content/drive/MyDrive/gdrive/heart-sound-database.zip'

In [ ]:
# !cp {'zip_path'}

In [ ]:
# !unzip -q 'heart-sound-database.zip'

In [ ]:
# data = os.path.join('/content/drive/MyDrive/gdrive/heart_sound')

In [ ]:
data = os.path.join('/kaggle/input/heart-sound-database/heart_sound')

In [ ]:
data

In [ ]:
train_dir_path = os.path.join(data, 'train')
valid_dir_path = os.path.join(data, 'val')

In [ ]:
train_dir_path

In [ ]:
valid_dir_path

In [ ]:
# pip install tensorflow-io

In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
healthy = os.path.join(train_dir_path, 'healthy', 'a0009.wav')
unhealthy = os.path.join(train_dir_path, 'unhealthy', 'a0001.wav')

In [ ]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels)
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
#     wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
wave = load_wav_16k_mono(healthy)
nwave = load_wav_16k_mono(unhealthy)

In [ ]:
plt.plot(nwave, color='red')
plt.plot(wave, color='blue')
plt.show()

In [ ]:
healthy_heart_train = tf.data.Dataset.list_files(train_dir_path+'/healthy'+'/*.wav')
unhealthy_heart_train = tf.data.Dataset.list_files(train_dir_path+'/unhealthy'+'/*.wav')

In [ ]:
healthy_heart_valid = tf.data.Dataset.list_files(valid_dir_path+'/healthy'+'/*.wav')
unhealthy_heart_valid = tf.data.Dataset.list_files(valid_dir_path+'/unhealthy'+'/*.wav')

In [ ]:
hhl_train = tf.data.Dataset.zip((healthy_heart_train, tf.data.Dataset.from_tensor_slices(tf.ones(len(healthy_heart_train)))))
uhl_train = tf.data.Dataset.zip((unhealthy_heart_train, tf.data.Dataset.from_tensor_slices(tf.zeros(len(unhealthy_heart_train)))))
train_data = hhl_train.concatenate(uhl_train)

In [ ]:
hhl_valid = tf.data.Dataset.zip((healthy_heart_valid, tf.data.Dataset.from_tensor_slices(tf.ones(len(healthy_heart_valid)))))
uhl_valid = tf.data.Dataset.zip((unhealthy_heart_valid, tf.data.Dataset.from_tensor_slices(tf.zeros(len(unhealthy_heart_valid)))))
valid_data = hhl_valid.concatenate(uhl_valid)

In [ ]:
train_data = train_data.concatenate(valid_data)

In [ ]:
lengths = []
for f in os.listdir(os.path.join(train_dir_path, 'healthy')):
    tensor_wave = load_wav_16k_mono(os.path.join(train_dir_path, 'healthy', f))
    lengths.append(len(tensor_wave))

In [ ]:
tf.math.reduce_mean(lengths)

In [ ]:
tf.math.reduce_min(lengths)

In [ ]:
tf.math.reduce_max(lengths)

In [ ]:
def preprocess(file_path, label):
    wav = load_wav_16k_mono(file_path)
    wav = wav[:40000]
    zero_padding = tf.zeros([40000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=80, frame_step=40)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [ ]:
iterator = hhl_train.shuffle(buffer_size=10000).as_numpy_iterator()

In [ ]:
iterator.next()

In [ ]:
try:
    filepath, label = next(iterator)  # Assuming it returns a tuple with two values
except StopIteration:
    # Handle the case where the iterator is empty
    print("Iterator is empty")

In [ ]:
spectrogram, label = preprocess(filepath, label)

In [ ]:
spectrogram

In [ ]:
plt.figure(figsize=(30,20))
plt.imshow(tf.transpose(spectrogram)[0])
plt.show()


In [ ]:
train_data = train_data.map(preprocess)
train_data = train_data.cache()
train_data = train_data.shuffle(buffer_size=1000)
train_data = train_data.batch(8)
train_data = train_data.prefetch(4)

In [ ]:
train_data

In [ ]:
num_samples = tf.data.experimental.cardinality(train_data).numpy()
print(f"Number of samples in train_data: {num_samples}")

In [ ]:
train = train_data.take(400)
test = train_data.skip(400).take(43)

In [ ]:
samples, labels = train.as_numpy_iterator().next()

In [ ]:
samples.shape

In [ ]:
labels

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), activation='relu', input_shape=(999, 65, 1)))
model.add(Conv2D(16, (3,3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('Adam', loss='BinaryCrossentropy', metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])

In [ ]:
model.summary()

In [ ]:
hist = model.fit(train, epochs=20, validation_data=test)

In [ ]:
plt.title('Loss')
plt.plot(hist.history['loss'], 'r')
plt.plot(hist.history['val_loss'], 'b')
plt.show()

In [ ]:
plt.title('Precision')
plt.plot(hist.history['precision'], 'r')
plt.plot(hist.history['val_precision'], 'b')
plt.show()

In [ ]:
plt.title('Recall')
plt.plot(hist.history['recall'], 'r')
plt.plot(hist.history['val_recall'], 'b')
plt.show()

In [ ]:
model.save('heartdisease_predictor_from_audio.h5')

In [ ]:
X_test, y_test = test.as_numpy_iterator().next()

In [ ]:
X_test.shape

In [ ]:
yhat = model.predict(X_test)

In [ ]:
yhat = [1 if prediction > 0.5 else 0 for prediction in yhat]

In [ ]:
print(y_test, yhat)